In [4]:
import pandas as pd
import geojson

In [8]:
elecbydist = pd.read_csv("./Data/election_results_by_state.csv")
elecbydist

,Year,State,state_fips,State_dem_votes,State_rep_votes
0,2016,Alabama,1,729541.0,1318249.0
1,2016,Alaska,2,0.0,0.0
2,2016,Arizona,4,1144854.0,1235770.0
3,2016,Arkansas,5,380490.0,684868.0
4,2016,California,6,8510070.0,4370301.0
...,...,...,...,...,...
95,2020,Virginia,51,2409153.0,1955598.0
96,2020,Washington,53,2369603.0,1584640.0
97,2020,West Virginia,54,235984.0,545382.0
98,2020,Wisconsin,55,1630662.0,1610052.0


In [9]:
years = [2016,2020]
for year in years:
    geofile = f"./Data/Geojson/ConDistricts{year}.geojson"
    with open(geofile,'r') as f:
        geodata = geojson.load(f)
        elecbydist = pd.read_csv("./Data/election_results_by_district.csv")
        elecbystate = pd.read_csv("./Data/election_results_by_state.csv")
        for feat in geodata['features']:
            proptoadd = {}
            samedist = elecbydist.loc[(elecbydist["Year"] == year)&(elecbydist["state_fips"]==int(feat.properties['STATEFP']))&(elecbydist["Congressional_District"]==feat.properties['NAMELSAD'])]
            if samedist.empty:
                proptoadd["WINNER"] = "Unknown"
                proptoadd["DEMVOTES"] = 0
                proptoadd["REPVOTES"] = 0
            else:
                if (samedist["District_dem_votes"].item() > samedist["District_rep_votes"].item()):
                    proptoadd["WINNER"] = "Democrat"
                    proptoadd["DEMVOTES"] = samedist["District_dem_votes"].item()
                    proptoadd["REPVOTES"] = samedist["District_rep_votes"].item()
                else:
                    proptoadd["WINNER"] = "Republican"
                    proptoadd["DEMVOTES"] = samedist["District_dem_votes"].item()
                    proptoadd["REPVOTES"] = samedist["District_rep_votes"].item()
            #Edit geojson with state election information
            samestate = elecbystate.loc[(elecbystate["Year"] == year)&(elecbystate["state_fips"]==int(feat.properties['STATEFP']))]
            if samestate.empty:
                proptoadd["STATENAME"] = "N/A"
                proptoadd["STATEWINNER"] = "Unknown"
                proptoadd["STATEDEMVOTES"] = 0
                proptoadd["STATEREPVOTES"] = 0
            else:
                if (samestate["State_dem_votes"].item() > samestate["State_rep_votes"].item()):
                    proptoadd["STATENAME"] = samestate["State"].item()
                    proptoadd["STATEWINNER"] = "Democrat"
                    proptoadd["STATEDEMVOTES"] = samestate["State_dem_votes"].item()
                    proptoadd["STATEREPVOTES"] = samestate["State_rep_votes"].item()
                else:
                    proptoadd["STATENAME"] = samestate["State"].item()
                    proptoadd["STATEWINNER"] = "Republican"
                    proptoadd["STATEDEMVOTES"] = samestate["State_dem_votes"].item()
                    proptoadd["STATEREPVOTES"] = samestate["State_rep_votes"].item()          
            feat.properties.update(proptoadd)
        with open(f"./Data/Geojson/RevConDistricts{year}.geojson",'w') as out:
            geojson.dump(geodata,out)
    #for feature in geodata: geodata[60].properties['NAMELSAD']
        